<a href="https://colab.research.google.com/github/HashiniDisanayaka/OOSD-TransMax/blob/master/PumpItUpDataMiningTheWaterTableMLchallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
#Import Required Modules

import gc
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [43]:
#Import data from csv files

training_value = pd.read_csv("/content/Training_set_values.csv")
test_value = pd.read_csv("/content/Test_set_values.csv")
training_label = pd.read_csv("/content/Training_set_labels.csv")

In [44]:
#PreProcess train label

column_label_value = list(training_label.columns.values)
column_label_value.remove("id")

for i in column_label_value:
	unique_value = training_label[i].unique()
	size = len(unique_value)
	print(size)
	for j in range(size):
		if unique_value[j] != "nan":
			training_label.loc[training_label[i] == unique_value[j], i] = j

3


In [45]:
#PreProcess training value and test csv files

column_label_value = list(training_value.columns.values)
column_label_value.remove("id")
column_label_value.remove("amount_tsh")
column_label_value.remove("gps_height")
column_label_value.remove("longitude")
column_label_value.remove("latitude")
column_label_value.remove("num_private")
column_label_value.remove("region_code")
column_label_value.remove("district_code")
column_label_value.remove("population")
column_label_value.remove("construction_year")

test_value = test_value.fillna(test_value.median())

for i in column_label_value:
	unique_value = list(set(np.concatenate((training_value[i].unique() , test_value[i].unique()))))
	size = len(unique_value)
	print(size)
	for j in range(size):
		if unique_value[j] != "nan":
			training_value.loc[training_value[i] == unique_value[j], i] = j
			test_value.loc[test_value[i] == unique_value[j], i] = j

training_value = training_value.fillna(training_value.median())
test_value = test_value.fillna(test_value.median())

369
2141
2411
45684
9
21426
21
125
2098
3
1
13
2869
3
18
13
7
12
5
7
7
8
6
5
5
10
7
3
7
6


In [46]:
#Combine Train value and Train lable files

train_data = training_value.merge(training_label, how = "outer", on = "id", sort = True)
train_data = train_data.fillna(train_data.median())


In [47]:
#Training process

# Feature Engineering
column_label_value = list(train_data.columns.values)
column_label_value.remove("id")
column_label_value.remove("date_recorded")
column_label_value.remove("status_group")
status_group = ["functional", "non functional", "functional needs repair"]

train_data = train_data.iloc[np.random.permutation(len(train_data))]

#Data Validation
amount = int(0.8*len(train_data))
validation = train_data[amount:]

#Classification using RandomForestClassifier
clasifier = RandomForestClassifier(n_estimators = 200, n_jobs = -1)

#Training data
clasifier.fit(train_data[column_label_value], train_data["status_group"])

#Calculating the accuracy
accuracy = accuracy_score(clasifier.predict(validation[column_label_value]), validation["status_group"])
print("Accuracy = " + str(accuracy))

#Freeing the RAM
del train_data, validation
gc.collect()

Accuracy = 1.0


48

In [48]:
#Testing process

#Testing data
test_value = test_value.fillna(test.median())

# Prediction on test data
prediction = clasifier.predict(test_value[column_label_value])

In [49]:
#Saving data into Submission.csv file
submission = pd.DataFrame({
			"id": test_value["id"],
			"status_group": prediction
		})
for i in range(len(status_group)):
	submission.loc[submission["status_group"] == i, "status_group"] = status_group[i]

#Save dataframe into the csv file
submission.to_csv("/content/Submission.csv", index = False)